In [1]:
from mpl_toolkits.basemap import Basemap  # import Basemap matplotlib toolkit
import numpy as np
import matplotlib.pyplot as plt
import pygrib # import pygrib interface to grib_api
import pandas as pd
import os
import sys
import math
import folium
import csv 

%matplotlib inline

root = os.getcwd()
root_data = root + "/grib_data"
root_URL = "http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/"
year = "2018"
month = "01"
day = "01"

def getgrib(year,month,day):
    file_name = "Z__C_RJTD_"+str(year+month+day)+"000000_MSM_GPV_Rjp_Lsurf_FH00-15_grib2.bin"
    file_url = root_URL\
                +str(year + "/" + month + "/" + day +"/" )\
                +file_name
    root_file= "grib_data/"+file_name
    if os.path.isfile(root_file) == False:
            command = "wget -P grib_data/ "+ file_url
            os.system(command)

    getgrib_output = pygrib.open(root_file)
    return getgrib_output

def getrootgrib(year,month,day):
    file_name = "Z__C_RJTD_"+str(year+month+day)+"000000_MSM_GPV_Rjp_Lsurf_FH00-15_grib2.bin"    
    return "grib_data/"+file_name

In [2]:
print(os.path.isfile( getrootgrib(year,month,day) ))

False


In [3]:
grbs = getgrib(year,month,day)

OSError: [Errno could not open %s] grib_data/Z__C_RJTD_20180101000000_MSM_GPV_Rjp_Lsurf_FH00-15_grib2.bin

In [ ]:
print(os.path.isfile( getrootgrib(year,month,day) ))

In [ ]:
#check parameters in gribfile
for grb in grbs:
    print(grb)

In [ ]:
# lats,lonsは二次元配列で緯度経度が入っている
lats, lons = grb.latlons()

#一次元に変換
# 二次元のままでは描画できない
# 2次元の格子点を1次元配列にならす
flat_lats= np.ravel(lats)
flat_lons= np.ravel(lons)

pd_all= {'lons':flat_lons, 'lats':flat_lats}

#ID 27 is Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 2 hrs:from 201801010000
# selectメソッドで指定した要素(forecastTime=0)にマッチする要素の配列が返ってくる。
# forecastTime=0の要素は複数あるが先頭([0])はTempratureが入っている

for i in range(0,6):
    grb = grbs.select(forecastTime=i)[5]
    # 中に含まれているデータ(temprature)を二次元のndarrayで取り出す
    temperature = grb.values
    #華氏を摂氏に変換
    flat_temperature= (np.ravel(temperature)-32)/1.8
    pd_name="temperature FH"+str(i)
    pd_all[pd_name]=flat_temperature
    
df_all= pd.DataFrame(pd_all)
print(len(df_all))
df_all[:10]

In [ ]:
#緯度経度の範囲を選択
east_lon=131.775
west_lon=130.0
north_lat=34
south_lat=33.0

In [ ]:
#range adjust
df=df_all[df_all['lons'] > west_lon]
df=df[df['lons'] < east_lon]

df=df[df['lats'] > south_lat]
df=df[df['lats'] < north_lat]
print(len(df))
df[:10]

In [ ]:
map_lat=df['lats'].values
map_lon=df['lons'].values
FH0=np.round(df['temperature FH0'].values,2)
FH1=np.round(df['temperature FH1'].values,2)
FH2=np.round(df['temperature FH2'].values,2)
FH3=np.round(df['temperature FH3'].values,2)
FH4=np.round(df['temperature FH4'].values,2)
FH5=np.round(df['temperature FH5'].values,2)

In [ ]:
from folium import plugins
#lat-long of focus area and zoom start
map = folium.Map([33.518022,130.471583], zoom_start=12)

# Mapbox makes some nice map tiles,
# Stamen also produce some cool map tiles which typically work at all zoom levels.
tile = folium.TileLayer('Stamen Terrain').add_to(map)

#adding marker and popup of temperature
for i in range(0,len(map_lat)):
    
    icon_number = plugins.BeautifyIcon(
    border_color='#00ABDC',
    text_color='#00ABDC',
    border_width=2,
    number=str(int(FH0[i])),
    inner_icon_style='margin-top:0;')
    
    folium.Marker(location=[float(map_lat[i])  ,float(map_lon[i])],\
                  popup="___temperature___"+"\n"\
                          +"__FH0= "+str(FH0[i])+"__\n"\
                          +"__FH1 = "+str(FH1[i])+"\__n"\
                          +"__FH2 = "+str(FH2[i])+"\__n"\
                          +"__FH3 = "+str(FH3[i])+"\__n"\
                          +"__FH4 = "+str(FH4[i])+"\__n"\
                          +"__FH5 = "+str(FH5[i])+"\__n",\
                 icon=icon_number).add_to(map)
    
#we can change tiles with help of LayerConrol
folium.LayerControl().add_to(map)

map_name="FH_Temperature Map: " + year + "/" + month + "/" + day
map.save(outfile="map_name")
map